In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import datetime
from config import props, filenames
try:
    import eclabfiles as ecf
except ImportError:
    print("eclabfiles not available")

In [ ]:
def read_CV_text(ecf_text_filename: str):
    with open(ecf_text_filename) as file:
        file.readline()
        Nb_header_lines = int(file.readline().split(' : ')[-1])
    ecfdf = pd.read_csv(ecf_text_filename, decimal=',', names=['t', 'U', 'I', 'cycle number'],
                        sep='\t', skiprows=Nb_header_lines, encoding='ANSI')

    return ecfdf

In [ ]:
def read_CV_mpr():
    data = ecf.to_df(f'{filenames.data}{props.codename}{props.CV_file}_C02.mpr')
    columns_dict = {
        'time'  : 't',
        'Ewe'   : 'U',
        '<I>'   : 'I',
        'cycle number' : 'cycle number',
    }
    data = pd.DataFrame(data, columns=columns_dict)
    data = data.rename(columns=columns_dict)
    return data

In [ ]:
data = read_CV_text(f'{filenames.data}{props.codename}{props.CV_file}_C02.txt')

In [ ]:
data = read_CV_mpr()

In [ ]:
total_nc = int(data['cycle number'].iloc[-1])
fig = plt.figure()
plt.title(f'CV for {props.codename}{props.CV_file} {total_nc = }')
plt.xlabel('U [V]')
plt.ylabel('J [mA/cm$^2$]')
plt.axhline(0.3, c='gray', ls='--')
plt.axhline(0.7, c='gray', ls='--')
for i in range(1, total_nc+1):
    cycle = data[data['cycle number'] == i]
    plt.plot(cycle.U, 100*cycle.I/props.el_area, color=plt.cm.viridis(i/total_nc))

In [ ]:
if not os.path.exists(filenames.notes):
    os.makedirs(filenames.notes)
if os.path.exists(f'{filenames.notes}/CV.png'):
    overwrite = (input('Do you want to overwrite the existing CV plot? [y/n]') in ('y', 'Y'))
    if overwrite:
        fig.savefig(f'{filenames.notes}/CV.png')
else:
    fig.savefig(f'{filenames.notes}/CV.png')